In [1]:
import pandas as pd
# df = pd.read_csv(r'C:\Users\mehar\Desktop\MIDAS\Internship\data\icdm_contest_data.csv')
df = pd.read_csv(r'C:\Users\mehar\Desktop\MIDAS\Internship\data\icdm_contest_data.csv', usecols = ['index', 'content', 'industry'])

In [2]:
df.head()

,index,content,industry
0,0,Norway has a lot of electric cars—so many that...,automotiveEngineering
1,1,Ford says shifter cables can snap off and rend...,automotiveEngineering
2,2,"Enlarge ImageThe C-Class is still mighty good,...",automotiveEngineering
3,3,The 2019 Cadillac CTS-VCNBC | Mack HoganCadill...,automotiveEngineering
4,4,"featuring 30-inch wheels, a dettatched cabin, ...",automotiveEngineering


In [3]:
print(len(df[df['industry'].str.contains('automotiveEngineering')]))
print(len(df[df['industry'].str.contains('publicSecurity')]))
print(len(df[df['industry'].str.contains('cosmetics')]))
print(len(df[df['industry'].str.contains('cateringServices')]))

90
70
69
71


In [4]:
df_domain_1 = df.loc[(df.industry == 'automotiveEngineering')]

In [5]:
import nltk
from nltk import tokenize

In [6]:
final_list  = []
l = len(df_domain_1)
for i in range(l):
    sent = df_domain_1['content'].iloc[i]
    final_list.append(tokenize.sent_tokenize(sent))

In [7]:
final_list

[['Norway has a lot of electric cars—so many that it can make anyone driving a new vehicle with an internal combustion engine look like a Luddite.',
  'Mercedes-Benz brought us there to experience the EQC—and possibly to normalize it in a sea of EVs that makes California look like a land of late adopters.',
  'Outside Oslo, where cars were larger and more upscale than in other parts of Europe, and Tesla vehicles (S and X) are a more common sight than around Los Angeles or the Bay Area, the EQC fit right in.',
  "DON'T MISS: Mercedes-Benz EQC Edition 1886 electric SUV kicks off a new era\r\nAfter a couple of rain-soaked days driving the EQC there last week, we can say that it will be a great addition in the U.S. when it arrives sometime in 2020.",
  'At about 187 inches long, the EQC400 4Matic crossover splices into the American mid-sizers.'],
 ["Ford says shifter cables can snap off and render the gear selector broken or useless on 2013–2016 Ford Fusion sedans.The automaker's latest re

In [194]:
import nltk, pandas as pd, numpy as np
from nltk.parse.corenlp import CoreNLPParser, CoreNLPDependencyParser
from nltk.tree import ParentedTree

In [195]:
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')
pos_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='pos')

In [196]:
def triplet_extraction (input_sent, output=['parse_tree','spo','result']):
    # Parse the input sentence with Stanford CoreNLP Parser
#     input_sent = tokenize.sent_tokenize(input_sent)
#     print("Tokenised", input_sent)
    pos_type = pos_tagger.tag(input_sent.split())
    parse_tree, = ParentedTree.convert(list(pos_tagger.parse(input_sent.split()))[0])
    dep_type, = ParentedTree.convert(dep_parser.parse(input_sent.split()))
    # Extract subject, predicate and object
    subject = extract_subject(parse_tree)
    predicate = extract_predicate(parse_tree)
    objects = extract_object(parse_tree)
#     if 'parse_tree' in output:
#         print('---Parse Tree---')
#         parse_tree.pretty_print()
#     if 'spo' in output:
#         print('---Subject---')
#         print(subject)
#         print('---Predicate---')
#         print(predicate)
#         print('---Object---')
#         print(objects)
    if 'result' in output:
        ans = ' '.join([subject[0], predicate[0], objects[0]])
        return ans
#         print('---Result---')
#         print(' '.join([subject[0], predicate[0], objects[0]]))

In [197]:
def extract_subject (parse_tree):
    # Extract the first noun found in NP_subtree
    subject = []
    for s in parse_tree.subtrees(lambda x: x.label() == 'NP'):
        for t in s.subtrees(lambda y: y.label().startswith('NN')):
            output = [t[0], extract_attr(t)]
            # Avoid empty or repeated values
            if output != [] and output not in subject:
                subject.append(output) 
    if len(subject) != 0: return subject[0] 
    else: return ['']

In [198]:
def extract_predicate (parse_tree):
    # Extract the deepest(last) verb foybd ub VP_subtree
    output, predicate = [],[]
    for s in parse_tree.subtrees(lambda x: x.label() == 'VP'):
        for t in s.subtrees(lambda y: y.label().startswith('VB')):
            output = [t[0], extract_attr(t)]
            if output != [] and output not in predicate:    
                predicate.append(output)
    if len(predicate) != 0: return predicate[-1]
    else: return ['']

In [199]:
def extract_object (parse_tree):
    # Extract the first noun or first adjective in NP, PP, ADP siblings of VP_subtree
    objects, output, word = [],[],[]
    for s in parse_tree.subtrees(lambda x: x.label() == 'VP'):
        for t in s.subtrees(lambda y: y.label() in ['NP','PP','ADP']):
            if t.label() in ['NP','PP']:
                for u in t.subtrees(lambda z: z.label().startswith('NN')):
                    word = u          
            else:
                for u in t.subtrees(lambda z: z.label().startswith('JJ')):
                    word = u
            if len(word) != 0:
                output = [word[0], extract_attr(word)]
            if output != [] and output not in objects:
                objects.append(output)
    if len(objects) != 0: return objects[0]
    else: return ['']

In [200]:
def extract_attr (word):
    attrs = []     
    # Search among the word's siblings
    if word.label().startswith('JJ'):
        for p in word.parent(): 
            if p.label() == 'RB':
                attrs.append(p[0])
    elif word.label().startswith('NN'):
        for p in word.parent():
            if p.label() in ['DT','PRP$','POS','JJ','CD','ADJP','QP','NP']:
                attrs.append(p[0])
    elif word.label().startswith('VB'):
        for p in word.parent():
            if p.label() == 'ADVP':
                attrs.append(p[0])
    # Search among the word's uncles
    if word.label().startswith('NN') or word.label().startswith('JJ'):
        for p in word.parent().parent():
            if p.label() == 'PP' and p != word.parent():
                attrs.append(' '.join(p.flatten()))
    elif word.label().startswith('VB'):
        for p in word.parent().parent():
            if p.label().startswith('VB') and p != word.parent():
                attrs.append(' '.join(p.flatten()))
    return attrs

In [201]:
longform = pd.DataFrame(columns=['content', 'industry'])
for idx, index, content, industry in df_domain_1.itertuples():
    content_words = tokenize.sent_tokenize(content)
    
    longform = longform.append(
        [{'content': nw, 'index':index, 'industry': industry} for nw in content_words],
        ignore_index = True
    )

longform['index'] = longform['index'].astype(int)
longform = longform[['index', 'content', 'industry']]
# print(longform['c'].iloc[1])
longform

,index,content,industry
0,0,Celebrity chef Jamie Oliver's British restaura...,cateringServices
1,0,The firm said Tuesday that it had gone into ad...,cateringServices
2,0,The company had been seeking buyers amid incre...,cateringServices
3,0,Oliver began his restaurant empire in 2002 whe...,cateringServices
4,0,"Oliver, known around the world for his cookboo...",cateringServices
5,0,"""He said ""I appreciate how difficult this is f...",cateringServices
6,0,I’m devastated that our much-loved UK restaura...,cateringServices
7,1,Celebrity chef Jamie Oliver's restaurant empir...,cateringServices
8,1,The entrepreneur and anti-junk food campaigner...,cateringServices
9,1,"The group includes Jamie's Italian chain, Jami...",cateringServices


In [202]:
longform

,index,content,industry
0,0,Celebrity chef Jamie Oliver's British restaura...,cateringServices
1,0,The firm said Tuesday that it had gone into ad...,cateringServices
2,0,The company had been seeking buyers amid incre...,cateringServices
3,0,Oliver began his restaurant empire in 2002 whe...,cateringServices
4,0,"Oliver, known around the world for his cookboo...",cateringServices
5,0,"""He said ""I appreciate how difficult this is f...",cateringServices
6,0,I’m devastated that our much-loved UK restaura...,cateringServices
7,1,Celebrity chef Jamie Oliver's restaurant empir...,cateringServices
8,1,The entrepreneur and anti-junk food campaigner...,cateringServices
9,1,"The group includes Jamie's Italian chain, Jami...",cateringServices


In [203]:
result_df = longform[['index', 'industry']]
result_tuple = []

result_Subject = []
result_Object =[]
result_Verb = []

for i in range(len(longform)):
    sent = longform['content'].iloc[i]
    result = triplet_extraction(sent)
    print(i, result)
    
    result_tuple.append(result)

0 Celebrity putting chain
1 firm operates protection
2 company according buyers
3 Oliver opened empire
4 Oliver put shows
5 everyone affected everyone
6 UK gone restaurants
7 Celebrity collapsed 
8 entrepreneur placed business
9 group train London
10 KPMG closed administrator
11 Jamie added term
12 Oliver run 
13 roundup is news
14 Prime is Bar
15 steakhouse offer tequila
16 Margarita leaves liquors
17 Baccarat take Baccarat
18   
19 Crazy offering tacos
20 Choice  
21 food opened restaurant
22 Petra operated menudo
23 addition includes quesadillas
24 Whittier opened Hollywood
25 restaurant utilizes cooking
26 Crary called Tree
27 Sunset is Venice
28 Shrimp rules ceviches
29 RestaurantData.com released Report
30 Concepts grown units
31 MIKE increased unit
32 sandwich is dinner
33 restaurants have seating
34 Catering is 
35 locations called unit
36 company are Italiana
37 company was growth
38 concepts are dinner
39 South said notice
40 DHEC tested May
41 Customers exposed May
42 DHEC a

In [204]:
result_tuple = [x for x in result_tuple if str(x) != 'nan']
result_tuple.append(np.nan)
# print(result_tuple)
for row in longform.itertuples():
    result_domain1 = result_df.append([{'index':index, 'industry': industry}],
        ignore_index = False)
print(len(result_domain1))
result_domain1['TripletsExtraction'] = result_tuple
result_domain1.dropna()
result_domain1

398


,index,industry,TripletsExtraction
0,0,cateringServices,Celebrity putting chain
1,0,cateringServices,firm operates protection
2,0,cateringServices,company according buyers
3,0,cateringServices,Oliver opened empire
4,0,cateringServices,Oliver put shows
5,0,cateringServices,everyone affected everyone
6,0,cateringServices,UK gone restaurants
7,1,cateringServices,Celebrity collapsed
8,1,cateringServices,entrepreneur placed business
9,1,cateringServices,group train London


In [205]:
result_domain1[['s1','r', 's2']] = result_domain1.TripletsExtraction.str.split(expand=True)

In [206]:
result_domain1 = result_domain1.join(longform[['content']])

In [207]:
result_domain1.to_csv("Result_Domain4.csv", index=False)